# Predicción de MP2.5 usando LSTM con Features Enriquecidas

In [1]:

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt


/Users/jimmyibanez/Projects/tesis_efg3_jimmy/tesis_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Cargar tus datos aquí (reemplaza con tus rutas de CSV)

In [2]:
def convertir_fechas(df):
    # Asegura que Timestamp_Pronostico sea datetime completo
    df['Timestamp_Pronostico'] = pd.to_datetime(df['Timestamp_Pronostico'])

    # Asegura que Fecha sea solo fecha (sin hora)
    df['Fecha'] = pd.to_datetime(df['Fecha']).dt.date

    return df

In [3]:
pronostico_met_df_2022 = pd.read_csv('pronostico_consolidado_2022_Curico.csv')
pronostico_met_df_2023 = pd.read_csv('pronostico_consolidado_2023_Curico.csv')
pronostico_met_df_2024 = pd.read_csv('pronostico_consolidado_2024_Curico.csv')

pronostico_co_df_2022 = pd.read_csv('pronostico_consolidado_completo_CO_2022.csv')
pronostico_co_df_2023 = pd.read_csv('pronostico_consolidado_completo_CO_2023.csv')
pronostico_co_df_2024 = pd.read_csv('pronostico_consolidado_completo_CO_2024.csv')


In [4]:
pronostico_mp25_baseline = pd.read_csv("df_pronostico_mp25.csv")

In [5]:
pronostico_met_df = pd.concat([pronostico_met_df_2022, pronostico_met_df_2023, pronostico_met_df_2024])
pronostico_co_df = pd.concat([pronostico_co_df_2022, pronostico_co_df_2023, pronostico_co_df_2024])

In [6]:
pronostico_met_df = convertir_fechas(pronostico_met_df)
pronostico_co_df = convertir_fechas(pronostico_co_df)

In [7]:
df_pronosticos = pd.merge(
    pronostico_met_df,
    pronostico_co_df,
    on=['Fecha', 'Timestamp_Pronostico', 'Region'],  # también es clave la Región
    how='inner'  # puedes cambiar a 'left', 'outer' o 'right' según el análisis
)
print(len(df_pronosticos))

72263


In [8]:
def limpiar_pronosticos(df, need_cut=True, first_val=-1):
  df = df.sort_values(by="Timestamp_Pronostico")
  if need_cut:
    df = df[4:-3]
  df['horizonte'] = [i for i in range(first_val, 4) for _ in range(24)]
  return df

In [9]:
df_pronosticos = df_pronosticos.groupby('Fecha').apply(limpiar_pronosticos).reset_index(drop=True)

/var/folders/_3/lz7prpr562j68q4k459330180000gp/T/ipykernel_20023/1442696180.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_pronosticos = df_pronosticos.groupby('Fecha').apply(limpiar_pronosticos).reset_index(drop=True)


In [10]:
observados_df = pd.read_csv('datos_observados_curico.csv', index_col=0)[['Timestamp', 'TEMP', 'WDIR', 'RHUM', 'WSPD', 'MP25_validado', 'MP25_preliminar', 'MP25_no_validado']]

In [11]:
# convertir a utc-4
observados_df['Timestamp'] = pd.to_datetime(observados_df['Timestamp'])
# observados_df['Timestamp'] = observados_df['Timestamp'].dt.tz_localize(pytz.utc)
# observados_df['Timestamp'] = observados_df['Timestamp'].dt.tz_convert('America/La_Paz') # (utc-4)

In [12]:
observados_df.head(3)

,Timestamp,TEMP,WDIR,RHUM,WSPD,MP25_validado,MP25_preliminar,MP25_no_validado
0,2022-01-01 01:00:00,15.1225,184.449,0.237583,1.86197,3.0,NaN,NaN
1,2022-01-01 02:00:00,14.0492,180.856,0.238750,1.48135,5.0,NaN,NaN
2,2022-01-01 03:00:00,13.1550,171.097,0.235000,1.09304,5.0,NaN,NaN


In [13]:
# Asegurarse de que esté ordenado cronológicamente
observados_df = observados_df.sort_values('Timestamp')

# Calcular media móvil de 24h para MP25_validado
def mm(df, columna, decimals=0, ventana=24):
    def promedio_personalizado(x):
        no_nulos = x.dropna()
        if len(x) - len(no_nulos) < 7:
            return round(no_nulos.mean(), decimals)
        else:
            return None

    resultado = df[columna].rolling(window=ventana, min_periods=1).apply(promedio_personalizado, raw=False)
    return resultado

# Aplicar función y crear columna MP25_mm
observados_df['MP25_mm'] = mm(observados_df, 'MP25_validado', decimals=2)


TypeError: must be real number, not NoneType

## Feature Engineering con variables compuestas

In [13]:
# Asegurar que 'Timestamp_Pronostico' sea datetime
df_pronosticos['Timestamp_Pronostico'] = pd.to_datetime(df_pronosticos['Timestamp_Pronostico'])

# Extraer componentes temporales
df_pronosticos['hour'] = df_pronosticos['Timestamp_Pronostico'].dt.hour
df_pronosticos['dayofweek'] = df_pronosticos['Timestamp_Pronostico'].dt.dayofweek
df_pronosticos['dayofyear'] = df_pronosticos['Timestamp_Pronostico'].dt.dayofyear

# Codificación cíclica
df_pronosticos['hour_sin'] = np.sin(2 * np.pi * df_pronosticos['hour'] / 24)
df_pronosticos['hour_cos'] = np.cos(2 * np.pi * df_pronosticos['hour'] / 24)

df_pronosticos['dayofweek_sin'] = np.sin(2 * np.pi * df_pronosticos['dayofweek'] / 7)
df_pronosticos['dayofweek_cos'] = np.cos(2 * np.pi * df_pronosticos['dayofweek'] / 7)

df_pronosticos['dayofyear_sin'] = np.sin(2 * np.pi * df_pronosticos['dayofyear'] / 365)
df_pronosticos['dayofyear_cos'] = np.cos(2 * np.pi * df_pronosticos['dayofyear'] / 365)

# Dirección del viento como ángulo cíclico
df_pronosticos['WDIR_sin'] = np.sin(np.deg2rad(df_pronosticos['WDIR']))
df_pronosticos['WDIR_cos'] = np.cos(np.deg2rad(df_pronosticos['WDIR']))


In [14]:

df_pronosticos['TEMP_WSPD'] = df_pronosticos['TEMP'] * df_pronosticos['WSPD']
df_pronosticos['CO_TEMP'] = df_pronosticos['CO(CENTRO)'] * df_pronosticos['TEMP']
df_pronosticos['CO_SO2'] = df_pronosticos['CO(CENTRO)'] / (df_pronosticos['SO2(CENTRO)'] + 1e-5)

feature_cols = [
    'TEMP', 'WSPD', 'WDIR_sin', 'WDIR_cos', 'CO(CENTRO)', 'SO2(CENTRO)',
    'hour_sin', 'hour_cos', 'dayofweek_sin', 'dayofweek_cos', 'dayofyear_sin', 'dayofyear_cos',
    'TEMP_WSPD', 'CO_TEMP', 'CO_SO2'
]


## Generación de secuencias de 24 horas

In [15]:

resultados = []
for fecha, grupo in df_pronosticos.groupby('Fecha'):
    grupo = grupo.sort_values('Timestamp_Pronostico').reset_index(drop=True)
    for idx, row in grupo.iterrows():
        if row['horizonte'] >= 0:
            ts = row['Timestamp_Pronostico']
            ventana = grupo[idx-24+1:idx+1]
            if len(ventana) == 24:
                resultados.append({
                    'Fecha': fecha,
                    'Timestamp_Pronostico': ts,
                    'Features': ventana[feature_cols].values,
                    'horizonte': row['horizonte']
                })

df_final = pd.DataFrame(resultados)


## Unión con MP2.5 observado y transformación logarítmica

In [16]:

df_modelo = df_final.merge(observados_df[['Timestamp', 'MP25_mm']], how='left', left_on='Timestamp_Pronostico', right_on='Timestamp')
df_modelo['MP25_log'] = np.log1p(df_modelo['MP25_mm'])

df_train = df_modelo[df_modelo['Timestamp_Pronostico'].dt.year.isin([2022, 2023])]
df_test = df_modelo[df_modelo['Timestamp_Pronostico'].dt.year == 2024]

X_train = np.stack(df_train['Features'].values)
y_train = df_train['MP25_log'].values
X_test = np.stack(df_test['Features'].values)
y_test = df_test['MP25_log'].values


KeyError: "['MP25_mm'] not in index"

## Escalado y generación de secuencias con Keras

In [ ]:

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[2])).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[2])).reshape(X_test.shape)

train_generator = TimeseriesGenerator(X_train_scaled, y_train, length=1, batch_size=32)
val_split = int(len(train_generator) * 0.8)
train_gen = TimeseriesGenerator(X_train_scaled[:val_split], y_train[:val_split], length=1, batch_size=32)
val_gen = TimeseriesGenerator(X_train_scaled[val_split:], y_train[val_split:], length=1, batch_size=32)


## Entrenamiento del modelo LSTM

In [ ]:

def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(32))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

model = build_lstm_model((X_train_scaled.shape[1], X_train_scaled.shape[2]))

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1),
    ModelCheckpoint("best_lstm_model.h5", monitor='val_loss', save_best_only=True, verbose=1)
]

history = model.fit(train_gen, validation_data=val_gen, epochs=100, callbacks=callbacks, verbose=1)
